https://www.evidentlyai.com/ranking-metrics/evaluating-recommender-systems

# set up

In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)          # or: client = QdrantClient(url="http://localhost:6333")

In [2]:
try:
    response = client.get_collections()
    print("Successfully connected to Qdrant.")
    print("Available collections:", response, sep="\n")
except Exception as e:
    print("Failed to connect to Qdrant:", e)

Successfully connected to Qdrant.
Available collections:
collections=[CollectionDescription(name='corpus_halong'), CollectionDescription(name='word-segmented-corpus_phobert'), CollectionDescription(name='word-segmented-corpus_phobert-trained')]


In [3]:
import os
import pandas as pd
import numpy as np
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.schema import TextNode

# evaluation function

In [4]:
def compute_MRR(retriever, test_set):
    mrr = 0
    for _, row in test_set.iterrows():
        retrieved_nodes = retriever.retrieve(row['question'])
        for j, node in enumerate(retrieved_nodes):
            if node.text == row['context']:
                mrr += 1/(j+1)
                break
    return mrr/len(test_set)

In [5]:
def compute_hit_rate(retriever, test_set):
    hit_rate = 0
    for _, row in test_set.iterrows():
        retrieved_nodes = retriever.retrieve(row['question'])
        for j, node in enumerate(retrieved_nodes):
            if node.text == row['context']:
                hit_rate += 1
                break
    return hit_rate/len(test_set)

In [19]:
def compute_ndcg(retriever, test_set):
    ndcg_values = []
    
    for _, row in test_set.iterrows():
        retrieved_nodes = retriever.retrieve(row['question'])
        relevance_scores = [1 if node.text == row['context'] else 0 for node in retrieved_nodes]

        if max(relevance_scores) == 0:  # No relevant document retrieved
            ndcg_values.append(0)
            continue

        # Compute DCG
        dcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(relevance_scores))

        # Compute IDCG (Ideal DCG) using sorted relevance scores
        ideal_relevance_scores = sorted(relevance_scores, reverse=True)
        idcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(ideal_relevance_scores))

        # Compute NDCG
        ndcg = dcg / idcg if idcg > 0 else 0
        ndcg_values.append(ndcg)

    return sum(ndcg_values)/len(test_set)


# word segmentation

In [7]:
corpus = pd.read_csv('corpus/corpus-seg.csv')
corpus.head(3)

,context,article,document
0,Điều 9 . Tuyển bổ_sung và loại ra khỏi chương_...,Điều 9 . Tuyển bổ_sung và loại ra khỏi chương_...,QUY_ĐỊNH ĐÀO_TẠO CHƯƠNG_TRÌNH TÀI_NĂNG
1,Điều 4 . Kiểm_tra xếp lớp đầu khóa cho sinh_vi...,Điều 4 . Kiểm_tra xếp lớp đầu khóa cho sinh_vi...,QUY_ĐỊNH ĐÀO_TẠO NGOẠI_NGỮ ĐỐI_VỚI HỆ ĐẠI_HỌC ...
2,Điều 5 . Chương_trình đào_tạo CT CLC được xâ...,Điều 5 . Chương_trình đào_tạo,QUY_ĐỊNH ĐÀO_TẠO CHƯƠNG_TRÌNH CHẤT_LƯỢNG CAO


In [8]:
nodes = [
    TextNode(
        text=row['context'],
        metadata={
            "article": row['article'],
            "document": row['document'],
        } 
    )
    for _, row in corpus.iterrows()
]

In [20]:
corpus_question_test_map = pd.read_csv("corpus/corpus-question-test-map-seg.csv")
corpus_question_test_map.head(3)

,context,article,document,question
0,"Điều 8 . Xây_dựng , thẩm_định học liệu điện_tử...","Điều 8 . Xây_dựng , thẩm_định học liệu điện_tử",QUY_ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG_THỨC TRỰC_TUYẾ...,Học_liệu điện_tử sau khi được thông_qua bởi ĐV...
1,"Điều 20 . Hồ_sơ , trình_tự , thủ_tục chỉnh_sửa...","Điều 20 . Hồ_sơ , trình_tự , thủ_tục chỉnh_sửa...","QUY_CHẾ Văn_bằng , chứng_chỉ của Trường Đại_họ...",Có_thể nộp bản_sao giấy khai_sinh cho hồ_sơ đề...
2,Điều 23 . Điểm_Miễn 1 . Điểm_BL - Sinh_viê...,Điều 23 . Điểm_Miễn,QUY_CHẾ ĐÀO_TẠO THEO HỌC CHẾ TÍN_CHỈ CHO HỆ ĐẠ...,Quy_định về điểm M trong trường_hợp sinh_viên ...


## phobert orginal

In [4]:
embed_model_phobert = HuggingFaceEmbedding(model_name="vinai/phobert-base-v2", max_length=256)

No sentence-transformers model found with name vinai/phobert-base-v2. Creating a new one with mean pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# QdrantVectorStore: docs.llamaindex.ai/en/stable/api_reference/storage/vector_store/qdrant/
qdrant_vector_store = QdrantVectorStore(client=client,
                                        collection_name="word-segmented-corpus_phobert",
                                        enable_hybrid=True)

storage_context = StorageContext.from_defaults(vector_store=qdrant_vector_store)

In [ ]:
# create index for the first time

index_phobert = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    embed_model=embed_model_phobert,
)

In [7]:
# Load the index from the existing vector store

index_phobert = VectorStoreIndex.from_vector_store(
    vector_store=qdrant_vector_store,
    storage_context=storage_context,
    embed_model=embed_model_phobert
)

In [8]:
retriever_phobert  = index_phobert.as_retriever(
    similarity_top_k=10,
    vector_store_query_mode="hybrid",
    alpha=0.5,
)

In [13]:
compute_hit_rate(retriever_phobert, corpus_question_test_map)

0.7161885245901639

In [21]:
compute_MRR(retriever_phobert, corpus_question_test_map)

0.4522256375227683

In [15]:
compute_ndcg(retriever_phobert, corpus_question_test_map)

0.5150766746563739

## phobert trained

In [24]:
# model_name can be either "path/to/fine-tuned model" or "KhoaUIT/Phobert-UIT-R2GQA"
embed_model_phobert_trained = HuggingFaceEmbedding(model_name="KhoaUIT/Phobert-UIT-R2GQA", max_length=256)

modules.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\AppData\Local\llama_index\models--KhoaUIT--Phobert-UIT-R2GQA. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/56.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/305 [00:00<?, ?B/s]

In [9]:
# QdrantVectorStore: docs.llamaindex.ai/en/stable/api_reference/storage/vector_store/qdrant/
qdrant_vector_store = QdrantVectorStore(client=client,
                                        collection_name="word-segmented-corpus_phobert-trained",
                                        enable_hybrid=True)

storage_context = StorageContext.from_defaults(vector_store=qdrant_vector_store)

In [13]:
# create index for the first time

index_phobert_trained = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    embed_model=embed_model_phobert_trained,
)

Collection word-segmented-corpus_phobert-trained already exists, skipping collection creation.


In [26]:
# Load the index from the existing vector store

index_phobert_trained = VectorStoreIndex.from_vector_store(
    vector_store=qdrant_vector_store,
    storage_context=storage_context,
    embed_model=embed_model_phobert_trained
)

In [27]:
retriever_phobert_trained  = index_phobert_trained.as_retriever(
    similarity_top_k=10,
    vector_store_query_mode="hybrid",
    alpha=0.5,
)

In [22]:
compute_hit_rate(retriever_phobert_trained, corpus_question_test_map)

0.9456967213114754

In [21]:
compute_MRR(retriever_phobert_trained, corpus_question_test_map)

0.7318232825917258

In [23]:
compute_ndcg(retriever_phobert_trained, corpus_question_test_map)

0.7843201524107308

# No word segmentation

In [8]:
corpus = pd.read_csv('corpus/corpus.csv')
corpus.head(3)

,context,article,document
0,Điều 9. Tuyển bổ sung và loại ra khỏi chương t...,Điều 9. Tuyển bổ sung và loại ra khỏi chương t...,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH TÀI NĂNG
1,Điều 4. Kiểm tra xếp lớp đầu khóa cho sinh viê...,Điều 4. Kiểm tra xếp lớp đầu khóa cho sinh viê...,QUY ĐỊNH ĐÀO TẠO NGOẠI NGỮ ĐỐI VỚI HỆ ĐẠI HỌC ...
2,Điều 5. Chương trình đào tạo CT CLC được xây d...,Điều 5. Chương trình đào tạo,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH CHẤT LƯỢNG CAO


In [9]:
nodes = [
    TextNode(
        text=row['context'],
        metadata={
            "article": row['article'],
            "document": row['document'],
        } 
    )
    for _, row in corpus.iterrows()
]

In [10]:
corpus_question_test_map = pd.read_csv("corpus/corpus-question-test-map.csv")
corpus_question_test_map.head(3)

,context,article,document,question
0,"Điều 8. Xây dựng, thẩm định học liệu điện tử 1...","Điều 8. Xây dựng, thẩm định học liệu điện tử",QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾ...,Học liệu điện tử sau khi được thông qua bởi ĐV...
1,"Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa nộ...","Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa nộ...","QUY CHẾ Văn bằng, chứng chỉ của Trường Đại học...",Có thể nộp bản sao giấy khai sinh cho hồ sơ đề...
2,Điều 23. Điểm Miễn 1. Điểm BL - Sinh viên đã t...,Điều 23. Điểm Miễn,QUY CHẾ ĐÀO TẠO THEO HỌC CHẾ TÍN CHỈ CHO HỆ ĐẠ...,Quy định về điểm M trong trường hợp sinh viên ...


## halong orginal

In [13]:
embed_model_halong = HuggingFaceEmbedding(model_name="hiieu/halong_embedding", max_length=512)

In [14]:
# QdrantVectorStore: docs.llamaindex.ai/en/stable/api_reference/storage/vector_store/qdrant/
qdrant_vector_store = QdrantVectorStore(client=client,
                                        collection_name="corpus_halong",
                                        enable_hybrid=True)

storage_context = StorageContext.from_defaults(vector_store=qdrant_vector_store)

In [16]:
# create index for the first time

index_halong = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    embed_model=embed_model_halong,
)

Collection corpus_halong already exists, skipping collection creation.


In [ ]:
# Load the index from the existing vector store

index_halong = VectorStoreIndex.from_vector_store(
    vector_store=qdrant_vector_store,
    storage_context=storage_context,
    embed_model=embed_model_halong
)

In [17]:
retriever_halong  = index_halong.as_retriever(
    similarity_top_k=10,
    vector_store_query_mode="hybrid",
    alpha=0.5,
)

In [19]:
compute_hit_rate(retriever_halong, corpus_question_test_map)

0.8780737704918032

In [20]:
compute_MRR(retriever_halong, corpus_question_test_map)

0.6312382090814463

In [21]:
compute_ndcg(retriever_halong, corpus_question_test_map)

0.6909850213699251

## halong trained

In [11]:
embed_model_halong_trained = HuggingFaceEmbedding(model_name="KhoaUIT/Halong-UIT-R2GQA", max_length=512)

modules.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\AppData\Local\llama_index\models--KhoaUIT--Halong-UIT-R2GQA. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/56.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/305 [00:00<?, ?B/s]

In [12]:
# QdrantVectorStore: docs.llamaindex.ai/en/stable/api_reference/storage/vector_store/qdrant/
qdrant_vector_store = QdrantVectorStore(client=client,
                                        collection_name="corpus_halong-trained",
                                        enable_hybrid=True)

storage_context = StorageContext.from_defaults(vector_store=qdrant_vector_store)

In [14]:
# create index for the first time

index_halong_trained = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    embed_model=embed_model_halong_trained,
)

Collection corpus_halong-trained already exists, skipping collection creation.


In [ ]:
# Load the index from the existing vector store

index_halong_trained = VectorStoreIndex.from_vector_store(
    vector_store=qdrant_vector_store,
    storage_context=storage_context,
    embed_model=embed_model_halong_trained
)

In [15]:
retriever_halong_trained  = index_halong_trained.as_retriever(
    similarity_top_k=10,
    vector_store_query_mode="hybrid",
    alpha=0.5,
)

In [16]:
compute_hit_rate(retriever_halong_trained, corpus_question_test_map)

0.9661885245901639

In [17]:
compute_MRR(retriever_halong_trained, corpus_question_test_map)

0.755805604345564

In [20]:
compute_ndcg(retriever_halong_trained, corpus_question_test_map)

0.808035384892791